## Functional Programming: Introduction

In [1]:
from pydantic import validate_call

Currying is often confused with partial functions. Currying is about transforming a function with *n* arguments into *n* functions with a single argument. 

In [2]:
# A helper function to conjugate Portuguese perfect tense
# Further below, we will use a more elegant way to do this.
def match_person(person):
    if person == "1.sg":
            return "i"
    elif person == "2.sg":
            return "ste"
    elif person == "3.sg":
            return "u"
    elif person == "1.pl":
            return "mos"
    elif person == "2.pl":
            return "stes"
    else:
            return "ram"

In [4]:
# A function with several inner functions
# Note that the variable "word" becomes part of the scope of the inner functions.
# This is called a closure. For functional programming, this is useful to keep
# state, without modifying the global state.
def conjugate_verb_a_stem_past(word):
    def add_tense_ablaut(tense_ablaut):
            def add_personal_ending(person):
                stem = word[0:-2]
                return stem + tense_ablaut + match_person(person)
            return add_personal_ending
    return add_tense_ablaut
    

In [5]:
conjugate_verb_a_stem_past("dar")("e")("1.pl")

'demos'

In [6]:
# We can define the above function simpler using a decorator
# for cleaner code
@curry
def conjugate_a_stem_past(word, tense_ablaut, person):
    stem = word[0:-2]
    return stem + tense_ablaut + match_person(person)

NameError: name 'curry' is not defined

In [ ]:
conjugate_a_stem_past("dar")("e")("1.pl")

'demos'

#### Dataclasses

*Dataclasses* allow us to define types. Think of those types as Python classes without methods attached to them.

##### Why Use Data Classes?

1. **Simplifies Class Definition**:
   - `dataclasses` automatically generate common methods like `__init__`, `__repr__`, `__eq__`, and others based on the class attributes. This reduces boilerplate code and makes class definitions more concise.

2. **Readability and Maintainability**:
   - By removing the need to write boilerplate code, `dataclasses` make class definitions cleaner and more focused on the data they represent. This enhances code readability and maintainability.

3. **Mutability Control**:
   - `dataclasses` provide control over mutability. You can define fields as immutable (using `frozen=True`) if needed, making instances of the dataclass hashable and usable as dictionary keys or set members.

4. **Default Values and Type Annotations**:
   - They support default values and type annotations, making it easier to define default states and document the expected types of each field. This leads to better code documentation and can help with static type checking.

5. **Easy Comparison and Ordering**:
   - With `dataclasses`, you can easily compare instances based on their field values without manually implementing the comparison methods. The `order=True` option adds comparison methods (`<`, `>`, `<=`, `>=`).

6. **Built-in Methods**:
   - Automatically provides useful methods like `asdict()` to convert the dataclass instance to a dictionary, making data manipulation and serialization straightforward.
 

By leveraging `dataclasses`, developers can focus more on the logic of their programs rather than on the repetitive details of class construction.


Below are two good examples on the kind of situations in which data classes are shine:
- https://www.youtube.com/watch?v=vBH6GRJ1REM
- https://www.youtube.com/watch?v=CvQ7e6yUtnw

In [8]:
import uuid
from dataclasses import dataclass

@dataclass 
class Student:
    name: str
    age: int
    id: uuid
    gpa: float

In [9]:
# You can instantiate a @dataclass position-wise
Student("niclas", 25, uuid.uuid4(), 19.4)

Student(name='niclas', age=25, id=UUID('d8099839-fa4a-4396-ba48-95d7e62878fa'), gpa=19.4)

In [10]:
# Or by instantiating it with keyword arguments
Student(**{"name": "niclas", "age": 25, "id": uuid.uuid4(), "gpa": 19.4})

Student(name='niclas', age=25, id=UUID('2d742884-015c-4860-b5d0-fd8b484bb2ed'), gpa=19.4)

#### Type Dispatching

**Type Dispatching** refers to the ability to execute different functions or methods based on the type of input arguments. This allows for more flexible and adaptable code, as the program can handle various data types or structures with appropriate logic.

#### Why Use Type Dispatching?
1. **Improves Code Flexibility**:
   - Allows functions to handle different types of inputs, making code more versatile and adaptable to various scenarios.
  
2. **Enhances Code Readability**:
   - By separating logic based on types, it becomes clearer what operations are performed on different data types.
  
3. **Encourages Reusability**:
   - You can write generic functions that work with multiple data types, reducing the need for type-specific functions.

4. **Reduces Complexity**:
   - Type dispatching can help avoid complex conditional statements (`if`/`elif` chains) to check the type of the input.

Python' *functools* library has a decorator called "singledispatch", which allows us to use functions of the same
name for different types of its first argument.

In [12]:
from functools import singledispatch

@dataclass
class Transaction:
    id: str
    amount: float
    sender: str
    receiver: str

@dataclass
class ForeignTransaction:
    id: str
    amount: float
    exchange_rate: float
    sender: str
    receiver: str

@dataclass
class Account:
    account_id: str
    balance: float

In [ ]:
@singledispatch
def process_transaction(transaction: Transaction, sender_account: Account, receiver_account: Account) -> List[Account]:    
    print(f"Processing transaction {transaction.id}")
    
    sender_account = Account(account_id=sender_account.account_id, balance=sender_account.balance - transaction.amount)
    receiver_account = Account(account_id=receiver_account.account_id, balance=receiver_account.balance + transaction.amount)
    
    print(f"Finished processiong transaction {transaction.id}")

    return sender_account, receiver_account  

# Each dispatched function must be identified. This can be a number or another identifier.
@process_transaction.register(ForeignTransaction) 
def _1(transaction: ForeignTransaction, sender_account: Account, receiver_account: Account) -> List[Account]:
    print(f"Processing transaction {transaction.id}")
    
    amount_sent = transaction.amount * transaction.exchange_rate
    
    sender_account = Account(account_id=sender_account.account_id, balance=sender_account.balance - amount_sent)
    receiver_account = Account(account_id=receiver_account.account_id, balance=receiver_account.balance + amount_sent)
    
    print(f"Finished processiong transaction {transaction.id}")

    return sender_account, receiver_account

#### Type Annotations (Hints)

Some programming languages enforce types. If you have a function that adds two numbers and both are integers, the compiler will not allow you to add two floats using that function. This is called type checking. In Python, 
we can annotate the types of variables and functions, but those annotations are not enforced.

Our primary objective is to write code that is *inspectable* and *debuggable*. Simple type errors can be easily avoided. 

In [13]:
def add(num1: int, num2: int) -> int:
    return num1 + num2

In [30]:
add(4.3, 5.4)

9.7

Using the **Pydantic** library, we can enforce type annotations. That is, we can specify the types of variables and when the function receives arguments different from those types, we get an error.

In [14]:
@validate_call
def add(num1: int, num2: int) -> int:
    return num1 + num2

In [15]:
# This will now result in an error
add(4.3, 5.4)

ValidationError: 2 validation errors for add
0
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=4.3, input_type=float]
    For further information visit https://errors.pydantic.dev/2.9/v/int_from_float
1
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=5.4, input_type=float]
    For further information visit https://errors.pydantic.dev/2.9/v/int_from_float

Note that we can also validate *dataclasses*. To do so, we need to override the *dataclass* namespace with a pydantic import.

In [53]:
from pydantic.dataclasses import dataclass

#### Exercise

Imagine you are writing a messaging application. Each user's inbox is represented by a *dataclass*. Each message is also represented by a *dataclass*. Write a function that allows you to send a message to a user.

*Hints*: Represent the inbox as a list of messages inside of a user's dataclass.

In [16]:
from typing import List
from uuid import UUID

@dataclass
class Message:
    content: str
    sender: UUID

@dataclass
class User:
    name: str
    user_id: UUID
    inbox: List[Message]

@validate_call
def send_message(user: User, message: Message) -> User:
    return User(name=user.name, user_id=user.user_id, inbox=user.inbox + [message])

In [17]:

# An empty list is a valid list, nothing will be validated
my_user = User(name="niclas", user_id=uuid.uuid4(), inbox=[])


In [18]:
another_user = User(name="ennius", user_id=uuid.uuid4(), inbox=[])

message = Message(content="Hello", sender=another_user.user_id)


In [19]:
my_user_updated = send_message(my_user, message)

In [20]:
my_user_updated

User(name='niclas', user_id=UUID('85860376-dd46-4abd-8720-1f0b24791606'), inbox=[Message(content='Hello', sender=UUID('e3569873-31c0-4a98-a300-d5b278274a3b'))])

#### Pattern Matching

In Python 3.10, *Pattern Matching* was introduced (to great controversy). In Functional Programming languages, Pattern Matching is very common and an established pattern. It allows us to decompose complex inputs into its constituents.

In [21]:
from datetime import datetime

In [22]:
data = {"time": datetime.now(), "customer_id": "123", "review": "This restaurant is the worst I've ever been to!"}

In [23]:
def process_string(string: str) -> List[str]:
    return string.lower().split(" ")

In [24]:
match data:
    case {"review": review}:
        processed = process_string(review)
        print(processed)

['this', 'restaurant', 'is', 'the', 'worst', "i've", 'ever', 'been', 'to!']


Remember the "sum" function we wrote in the beginning? We can express the recursive version of it using pattern matching.

In [25]:
def sum(ls):
    match ls:
        case []:
            return 0
        case _:
            return ls[0] + sum(ls[1:])